In [1]:
python_code = '''
import random
result = 'heads' if random.randint(0,1) == 0 else 'tails'
print(result)
'''

In [2]:
import kfp
from kfp import dsl

@dsl.pipeline(name='Sequential pipeline', description='A pipeline with two sequential steps.')
def sequential_pipeline():
    flip = flip_coin_op()
    print_op(flip.output)

def flip_coin_op():
    return dsl.ContainerOp(
        name='Flip coin',
        image='python:alpine3.6',
        command=['sh', '-c'],
        arguments=[f'python -c "{python_code}" | tee /tmp/output'],
        file_outputs={'output': '/tmp/output'}
    )

def print_op(msg):
    return dsl.ContainerOp(
        name='Print',
        image='alpine:3.6',
        command=['echo', msg],
    )

In [3]:
pipelineGzFile = 'hahahoho.zip'

kfp.compiler.Compiler().compile(
    sequential_pipeline,
    pipelineGzFile
)

client = kfp.Client()
my_exp = client.create_experiment(
    name='Basic Experiment'
)
my_run = client.run_pipeline(
    my_exp.id,
    'Sequential pipeline', 
    pipelineGzFile
)